In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
from math_rag.infrastructure.containers import InfrastructureContainer


RESET = False

# containers
infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()
infrastructure_container.wire(modules=[__name__])

application_container = infrastructure_container.application_container()
application_container.init_resources()
application_container.wire(modules=[__name__])

# seed
for object_seeder in infrastructure_container.object_seeders():
    object_seeder.seed(reset=RESET)

for document_seeder in infrastructure_container.document_seeders():
    await document_seeder.seed(reset=RESET)

for embedding_seeder in infrastructure_container.embedding_seeders():
    await embedding_seeder.seed(reset=RESET)

# index
for document_indexer in infrastructure_container.document_indexers():
    await document_indexer.index(reset=RESET)

2025-06-14 18:51:11,926 - INFO - PyTorch version 2.6.0 available.
2025-06-14 18:51:12,198 - INFO - HTTP Request: GET http://qdrant:6333 "HTTP/1.1 200 OK"
2025-06-14 18:51:12,202 - INFO - HTTP Request: GET http://qdrant:6333/collections/mathexpressiondescriptionembedding/exists "HTTP/1.1 200 OK"


In [4]:
fine_tune_job_runner_service = infrastructure_container.fine_tune_job_runner_service()

In [ ]:
await fine_tune_job_runner_service.init_resources()

In [6]:
from math_rag.core.models import FineTuneJob


fine_tune_job = FineTuneJob(provider_name='meta-llama', model_name='Llama-3.1-8B-Instruct')

In [ ]:
await fine_tune_job_runner_service.run(fine_tune_job, poll_interval=5 * 60)